In [185]:
from gravity_toolkit.read_GRACE_harmonics import read_GRACE_harmonics

Ylms1 = read_GRACE_harmonics("GSM-2_2004306-2004335_GRAC_UTCSR_BB01_0600", 85)
Ylms2 = read_GRACE_harmonics("GSM-2_2004336-2004366_GRAC_UTCSR_BB01_0600", 85)

data = {
    "l": Ylms1["l"],
    "m": Ylms1["m"],
    "clm1": Ylms1["clm"],
    "slm1": Ylms1["slm"],
    "clm2": Ylms2["clm"],
    "slm2": Ylms2["slm"],
}

#### Functions

In [60]:
from tqdm import tqdm
import numpy as np
from scipy.special import sph_harm


def compute_gravitational_anomaly(
    lon,
    lat,
    l_max,
    clm,
    slm,
    exclude_low_degrees,
    gm=3.986004415e14,
    earth_radius=6.3781363000e06,
):

    theta = np.radians(90 - lat)
    phi = np.radians(lon)

    g_theta = np.zeros_like(theta, dtype=np.float64)
    g_phi = np.zeros_like(theta, dtype=np.float64)
    g_r = np.zeros_like(theta, dtype=np.float64)

    for l in tqdm(range(exclude_low_degrees, l_max + 1)):
        for m in range(l + 1):
            ylm_real = sph_harm(m, l, phi, theta).real
            ylm_imag = sph_harm(m, l, phi, theta).imag

            plm_derivative = np.cos(theta) * sph_harm(m, l, phi, theta).real

            g_theta += clm[l, m] * plm_derivative + slm[l, m] * np.sin(theta) * ylm_imag
            g_phi += m * (-clm[l, m] * ylm_imag + slm[l, m] * ylm_real)
            g_r += (l + 1) * (clm[l, m] * ylm_real + slm[l, m] * ylm_imag)

    g_theta = -g_theta / earth_radius
    g_phi = -g_phi / (earth_radius * np.sin(theta))
    g_r = -g_r * gm / earth_radius**2

    g_h = np.sqrt(g_theta**2 + g_phi**2)

    g_total = np.sqrt(g_h**2 + g_r**2)

    g_total *= 1e8

    return g_total


def compute_power_spectrum(Clm, Slm, Lmax):
    power = np.zeros(Lmax + 1)
    for l in range(2, Lmax + 1):
        power[l] = np.sum(Clm[l, : l + 1] ** 2 + Slm[l, : l + 1] ** 2)
    return power

#### Full world map

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Parameters
l_max = 85
grid_size = 180
# Exclude low-degree terms to focus on localized anomalies
exclude_low_degrees = 70

clm1 = data["clm1"]
slm1 = data["slm1"]
clm2 = data["clm2"]
slm2 = data["slm2"]

lat = np.linspace(-90, 90, grid_size)
lon = np.linspace(0, 360, grid_size)
lon_grid, lat_grid = np.meshgrid(lon, lat)


# Compute total gravitational anomalies for two time periods
gravitational_anomalies1 = compute_gravitational_anomaly(
    l_max=l_max,
    lon=lon_grid,
    lat=lat_grid,
    clm=clm1,
    slm=slm1,
    exclude_low_degrees=exclude_low_degrees,
)
gravitational_anomalies2 = compute_gravitational_anomaly(
    l_max=l_max,
    lon=lon_grid,
    lat=lat_grid,
    clm=clm2,
    slm=slm2,
    exclude_low_degrees=exclude_low_degrees,
)

# Compute the difference between the two anomalies
grav_difference = np.abs(gravitational_anomalies2 - gravitational_anomalies1)

# Plot the gravitational anomalies
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={"projection": ccrs.PlateCarree()})
ax.set_global()
ax.add_feature(cfeature.COASTLINE)
c = ax.contourf(
    lon_grid,
    lat_grid,
    grav_difference,
    transform=ccrs.PlateCarree(),
    cmap="RdYlBu_r",
    levels=20,
)
plt.colorbar(c, ax=ax, label="Total Gravitational Anomaly Difference ($\mu$Gal)")
plt.title("Gravitational Anomaly Map")
plt.show()

## Localized Region

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


# Parameters
l_max = 85
grid_size = 180
# Exclude low-degree terms to focus on localized anomalies
exclude_low_degrees = 70

lat_center, lon_center = 42.35, 13.38
lat_range = 3
lon_range = 5

lat_min, lat_max = lat_center - lat_range, lat_center + lat_range
lon_min, lon_max = lon_center - lon_range, lon_center + lon_range

lat = np.linspace(lat_min, lat_max, grid_size)
lon = np.linspace(lon_min, lon_max, grid_size)
lon_grid, lat_grid = np.meshgrid(lon, lat)

clm1 = data["clm1"]
slm1 = data["slm1"]
clm2 = data["clm2"]
slm2 = data["slm2"]

# Compute total gravitational anomalies for two time periods
gravitational_anomalies1 = compute_gravitational_anomaly(
    l_max=l_max,
    lon=lon_grid,
    lat=lat_grid,
    clm=clm1,
    slm=slm1,
    exclude_low_degrees=exclude_low_degrees,
)
gravitational_anomalies2 = compute_gravitational_anomaly(
    l_max=l_max,
    lon=lon_grid,
    lat=lat_grid,
    clm=clm2,
    slm=slm2,
    exclude_low_degrees=exclude_low_degrees,
)

# Compute the difference between anomalies
grav_difference = np.abs(gravitational_anomalies2 - gravitational_anomalies1)

# Apply the mask to focus only on the region of interest
lat_indices = (lat >= lat_min) & (lat <= lat_max)
lon_indices = (lon >= lon_min) & (lon <= lon_max)

lon_subset = lon[lon_indices]
lat_subset = lat[lat_indices]
anomalies_subset = grav_difference[np.ix_(lat_indices, lon_indices)]

# Plot the gravitational anomalies
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={"projection": ccrs.PlateCarree()})
ax.set_extent(
    [lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree()
)
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.add_feature(cfeature.BORDERS, linestyle=":")
ax.add_feature(cfeature.LAND, edgecolor="black", facecolor="none")

ax.scatter(
    lon_center,
    lat_center,
    color="red",
    s=50,
    label="Epicenter",
    transform=ccrs.PlateCarree(),
    zorder=5,
)
ax.text(
    lon_center + 0.05,
    lat_center,
    "L'Aquila Epicenter",
    fontsize=9,
    color="red",
    transform=ccrs.PlateCarree(),
)

# Create a contour plot for the anomalies
c = ax.contourf(
    lon_subset,
    lat_subset,
    anomalies_subset,
    transform=ccrs.PlateCarree(),
    cmap="RdYlBu_r",
    levels=20,
)
plt.colorbar(c, ax=ax, label="Gravitational Anomaly ($\mu$Gal)")
plt.title("Gravitational Anomaly Map over Italy")
plt.show()

#### Anomaly at specific coordinates

In [ ]:
# Parameters
l_max = 85
exclude_low_degrees = 70
clm1 = data["clm1"]
slm1 = data["slm1"]
clm2 = data["clm2"]
slm2 = data["slm2"]

latitude = 34.45 
longitude = 73.60

# Compute the anomaly at specific coordinates
# gravitational_anomalies1 = compute_gravitational_anomaly(lat=latitude, lon=longitude, l_max=l_max, clm=clm1, slm=slm1, exclude_low_degrees=exclude_low_degrees)
gravitational_anomalies2 = compute_gravitational_anomaly(
    lat=latitude,
    lon=longitude,
    l_max=l_max,
    clm=clm2,
    slm=slm2,
    exclude_low_degrees=exclude_low_degrees,
)

print(
    f"Gravitational Anomaly at ({latitude}, {longitude}): {gravitational_anomalies2:.10f} µGal"
)

#### Power spectrum

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Lmax = 85
power_spectrum = compute_power_spectrum(clm1, slm1, Lmax)

# Plot power spectrum
plt.figure(figsize=(8, 6))
plt.semilogy(range(2, Lmax + 1), power_spectrum[2:], "o-", color="b")
plt.title("Spherical Harmonic Power Spectrum")
plt.xlabel("Spherical Harmonic Degree (l)")
plt.ylabel("Power (Clm² + Slm²)")
plt.grid(True)
plt.show()